In [37]:
import pandas as pd
import requests
import time
import os
from tqdm import tqdm  # Progress bar
import json

In [41]:
# Define API Key
API_KEY = "XdMGVdt2jC9VHVFG2HKdNPg0lBDBSdEepohce1Tt"

# Define input and output files
input_file = "unique_bills_with_congno.csv"  # The CSV file with bill numbers and congress numbers
output_file = "congress_bill_data.csv"  # The file where results will be saved

# API rate limits
# MAX_REQUESTS_PER_HOUR = 5000  # Hard limit from API
# REQUEST_DELAY = 0.5  # 750ms to ensure < 5000 requests per hour

In [42]:
# Load the CSV file
df = pd.read_csv(input_file)
print(f"Total bills to process: {len(df)}")

df.head()

Total bills to process: 21728


,Index,Bill_Name,CongNo
0,0,S.926,117
1,1,H.R.1324,117
2,2,S.467,117
3,3,H.R.1667,117
4,4,S.610,117


In [45]:
'''
def get_bill_type_and_number(bill_name):
    """
    Extracts bill type and number manually by splitting at the last period.
    Fixes cases like "H.R.1324" -> ("hr", "1324"), "S.CON.RES.11" -> ("sconres", "11").
    Ignores amendments like "S.AMDT.133" since they are not valid bill types.
    """
    bill_parts = bill_name.rsplit(".", 1)  # Split at last period
    if len(bill_parts) != 2:
        return None, None  # Invalid format

    bill_type_map = {
        "H.R": "hr",
        "S": "s",
        "H.J.RES": "hjres",
        "S.J.RES": "sjres",
        "H.CON.RES": "hconres",
        "S.CON.RES": "sconres",
        "H.RES": "hres",
        "H.R.RES": "hres",
        "S.RES": "sres",
        "S.R.RES": "sres"
    }

    bill_type = bill_parts[0].upper()  # Normalize case
    bill_number = bill_parts[1]

    # Check if bill type is valid
    return bill_type_map.get(bill_type, None), bill_number


def fetch_and_save_all_bills(df, filename):
    """
    Fetches bill data for all rows in the DataFrame while respecting API limits.
    """
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Bills", unit="bill"):
        bill_name = row["Bill_Name"]
        congress = row["CongNo"]
        
        # Extract bill type and number correctly
        bill_type, bill_number = get_bill_type_and_number(bill_name)
        if not bill_type or not bill_number:
            print(f"Skipping invalid bill name: {bill_name}")
            continue  # Skip if extraction fails

        # Construct API URL
        url = f"https://api.congress.gov/v3/bill/{congress}/{bill_type}/{bill_number}?api_key={API_KEY}"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json().get("bill", {})

            # Extract relevant fields
            bill_info = {
                "bill_number": data.get("number"),
                "congress_number": data.get("congress"),
                "bill_title": data.get("title"),
                "policy_area": data.get("policyArea", {}).get("name"),
                "introduced_date": data.get("introducedDate"),
            }

            # Process each sponsor as a separate row
            sponsors = data.get("sponsors", [])
            if not sponsors:  # If no sponsors, add a row with empty sponsor fields
                results.append({**bill_info, "sponsor_name": None, "sponsor_party": None, "sponsor_bioguide_id": None})
            else:
                for sponsor in sponsors:
                    results.append({
                        **bill_info,
                        "sponsor_name": sponsor.get("fullName"),
                        "sponsor_party": sponsor.get("party"),
                        "sponsor_bioguide_id": sponsor.get("bioguideId")
                    })

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {bill_name}: {e}")

        # Small delay to stay within API limits
        # time.sleep(REQUEST_DELAY)

    # Convert to DataFrame
    result_df = pd.DataFrame(results)

    # Append to CSV or create if it doesn’t exist
    if not result_df.empty:
        file_exists = os.path.exists(filename)
        result_df.to_csv(filename, mode="a", header=not file_exists, index=False)
        print(f"Data saved to {filename}.")
    else:
        print("No data retrieved.")
        '''

In [48]:
def get_bill_type_and_number(bill_name):
    """
    Extracts bill type and number manually by splitting at the last period.
    Handles cases like "H.R.1324" -> ("hr", "1324") and amendments "H.AMDT.133" -> ("hamdt", "133").
    Uses "bill" for regular bills and "amendment" for amendments in API calls.
    """
    bill_parts = bill_name.rsplit(".", 1)  # Split at last period
    if len(bill_parts) != 2:
        return None, None, None  # Invalid format

    bill_type_map = {
        "H.R": ("hr", "bill"),
        "S": ("s", "bill"),
        "H.J.RES": ("hjres", "bill"),
        "S.J.RES": ("sjres", "bill"),
        "H.CON.RES": ("hconres", "bill"),
        "S.CON.RES": ("sconres", "bill"),
        "H.RES": ("hres", "bill"),
        "S.RES": ("sres", "bill"),
        "H.R.RES": ("hres", "bill"),
        "S.R.RES": ("sres", "bill"),
        "H.AMDT": ("hamdt", "amendment"),
        "S.AMDT": ("samdt", "amendment")
    }

    bill_type = bill_parts[0].upper()  # Normalize case
    bill_number = bill_parts[1]

    # Check if bill type is valid
    return bill_type_map.get(bill_type, (None, None)) + (bill_name,)

In [51]:
def fetch_and_save_all_bills(df, filename):
    """
    Fetches bill and amendment data for all rows in the DataFrame while respecting API limits.
    """
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Bills", unit="bill"):
        bill_name = row["Bill_Name"]
        congress = row["CongNo"]
        
        # Extract bill type, API endpoint type, and full bill name
        bill_type, api_type, full_bill_code = get_bill_type_and_number(bill_name)
        if not bill_type or not api_type:
            print(f"Skipping invalid bill name: {bill_name}")
            continue  # Skip if extraction fails

        # Construct API URL
        url = f"https://api.congress.gov/v3/{api_type}/{congress}/{bill_type}/{bill_name.split('.')[-1]}?api_key={API_KEY}"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json().get(api_type, {})  # Extract data from the correct API type

            # Extract relevant fields
            bill_info = {
                "bill_number": data.get("number"),
                "congress_number": data.get("congress"),
                "bill_title": data.get("title"),
                "policy_area": data.get("policyArea", {}).get("name"),
                "introduced_date": data.get("introducedDate"),
                "bill_type": bill_type,  # hr, s, etc.
                "full_bill_code": full_bill_code  # H.R.123, S.456, etc.
            }

            # Process each sponsor as a separate row
            sponsors = data.get("sponsors", [])
            if not sponsors:  # If no sponsors, add a row with empty sponsor fields
                results.append({**bill_info, "sponsor_name": None, "sponsor_party": None, "sponsor_bioguide_id": None})
            else:
                for sponsor in sponsors:
                    results.append({
                        "bill_name": bill_name,
                        **bill_info,
                        "sponsor_name": sponsor.get("fullName"),
                        "sponsor_party": sponsor.get("party"),
                        "sponsor_bioguide_id": sponsor.get("bioguideId")
                    })

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {bill_name}: {e}")

        # Small delay to stay within API limits
        # time.sleep(REQUEST_DELAY)

        print(results)

    # Convert to DataFrame
    result_df = pd.DataFrame(results)

    # Append to CSV or create if it doesn’t exist
    if not result_df.empty:
        file_exists = os.path.exists(filename)
        result_df.to_csv(filename, mode="a", header=not file_exists, index=False)
        print(f"Data saved to {filename}.")
    else:
        print("No data retrieved.")

In [52]:
fetch_and_save_all_bills(df, output_file)

Processing Bills:   0%|          | 1/21728 [00:01<6:22:09,  1.06s/bill]

[{'bill_name': 'S.926', 'bill_number': '926', 'congress_number': 117, 'bill_title': 'SASCA', 'policy_area': 'Health', 'introduced_date': '2021-03-23', 'bill_type': 's', 'full_bill_code': 'S.926', 'sponsor_name': 'Sen. Murray, Patty [D-WA]', 'sponsor_party': 'D', 'sponsor_bioguide_id': 'M001111'}]


Processing Bills:   0%|          | 2/21728 [00:02<6:02:45,  1.00s/bill]

[{'bill_name': 'S.926', 'bill_number': '926', 'congress_number': 117, 'bill_title': 'SASCA', 'policy_area': 'Health', 'introduced_date': '2021-03-23', 'bill_type': 's', 'full_bill_code': 'S.926', 'sponsor_name': 'Sen. Murray, Patty [D-WA]', 'sponsor_party': 'D', 'sponsor_bioguide_id': 'M001111'}, {'bill_name': 'H.R.1324', 'bill_number': '1324', 'congress_number': 117, 'bill_title': 'Effective Suicide Screening and Assessment in the Emergency Department Act of 2021', 'policy_area': 'Health', 'introduced_date': '2021-02-25', 'bill_type': 'hr', 'full_bill_code': 'H.R.1324', 'sponsor_name': 'Rep. Bilirakis, Gus M. [R-FL-12]', 'sponsor_party': 'R', 'sponsor_bioguide_id': 'B001257'}]


Processing Bills:   0%|          | 3/21728 [00:02<5:53:38,  1.02bill/s]

...

[{'bill_name': 'S.926', 'bill_number': '926', 'congress_number': 117, 'bill_title': 'SASCA', 'policy_area': 'Health', 'introduced_date': '2021-03-23', 'bill_type': 's', 'full_bill_code': 'S.926', 'sponsor_name': 'Sen. Murray, Patty [D-WA]', 'sponsor_party': 'D', 'sponsor_bioguide_id': 'M001111'}, {'bill_name': 'H.R.1324', 'bill_number': '1324', 'congress_number': 117, 'bill_title': 'Effective Suicide Screening and Assessment in the Emergency Department Act of 2021', 'policy_area': 'Health', 'introduced_date': '2021-02-25', 'bill_type': 'hr', 'full_bill_code': 'H.R.1324', 'sponsor_name': 'Rep. Bilirakis, Gus M. [R-FL-12]', 'sponsor_party': 'R', 'sponsor_bioguide_id': 'B001257'}, {'bill_name': 'S.467', 'bill_number': '467', 'congress_number': 117, 'bill_title': 'Effective Suicide Screening and Assessment in the Emergency Department Act of 2021', 'policy_area': 'Health', 'introduced_date': '2021-02-25', 'bill_type': 's', 'full_bill_code': 'S.467', 'sponsor_name': 'Sen. Murkowski, Lisa [R-

Processing Bills:   0%|          | 5/21728 [00:04<5:58:58,  1.01bill/s]

[{'bill_name': 'S.926', 'bill_number': '926', 'congress_number': 117, 'bill_title': 'SASCA', 'policy_area': 'Health', 'introduced_date': '2021-03-23', 'bill_type': 's', 'full_bill_code': 'S.926', 'sponsor_name': 'Sen. Murray, Patty [D-WA]', 'sponsor_party': 'D', 'sponsor_bioguide_id': 'M001111'}, {'bill_name': 'H.R.1324', 'bill_number': '1324', 'congress_number': 117, 'bill_title': 'Effective Suicide Screening and Assessment in the Emergency Department Act of 2021', 'policy_area': 'Health', 'introduced_date': '2021-02-25', 'bill_type': 'hr', 'full_bill_code': 'H.R.1324', 'sponsor_name': 'Rep. Bilirakis, Gus M. [R-FL-12]', 'sponsor_party': 'R', 'sponsor_bioguide_id': 'B001257'}, {'bill_name': 'S.467', 'bill_number': '467', 'congress_number': 117, 'bill_title': 'Effective Suicide Screening and Assessment in the Emergency Department Act of 2021', 'policy_area': 'Health', 'introduced_date': '2021-02-25', 'bill_type': 's', 'full_bill_code': 'S.467', 'sponsor_name': 'Sen. Murkowski, Lisa [R-

Processing Bills:   0%|          | 5/21728 [00:05<6:53:58,  1.14s/bill]


KeyboardInterrupt: 